In [1]:
!pip install underthesea==6.8.4 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import re
from underthesea import sent_tokenize, word_tokenize

In [3]:
df = pd.read_csv("/kaggle/input/cogdis-data/Combined_Data.csv")
print(f"Loaded {len(df)} rows")

Loaded 20092 rows


In [4]:
def count_turns(text):
    if not isinstance(text, str): 
        return 1
    # Mẫu regex nhận diện lượt nói
    return len(re.findall(r"(?:^|\s)([A-ZĐƠÔƯỮẠ-ỹa-z]+:)", text)) or 1

df["turn_count"] = df["Original Text"].apply(count_turns)

In [5]:
question_words = ["tại sao", "phải chăng", "liệu", "sao", "vì sao", "có phải", "à", "ư", "hả", "chứ"]

def question_ratio(text):
    if not isinstance(text, str): return 0
    sentences = sent_tokenize(text)
    if not sentences: return 0
    question_sents = [s for s in sentences if "?" in s or any(q in s.lower() for q in question_words)]
    return round(len(question_sents) / len(sentences), 3)

df["question_ratio"] = df["Original Text"].apply(question_ratio)

In [6]:
discourse_markers = ["nhưng", "tuy nhiên", "mặc dù", "thế mà", "song", "trái lại", "dù vậy", "tuy thế"]

def discourse_marker_count(text):
    if not isinstance(text, str): return 0
    return sum(len(re.findall(rf"\b{w}\b", text.lower())) for w in discourse_markers)

df["discourse_marker_count"] = df["Original Text"].apply(discourse_marker_count)

In [7]:
self_refs = ["tôi", "mình", "bản thân tôi", "chính tôi", "tự tôi"]

def self_reference_count(text):
    if not isinstance(text, str): return 0
    return sum(len(re.findall(rf"\b{w}\b", text.lower())) for w in self_refs)

df["self_reference_count"] = df["Original Text"].apply(self_reference_count)

In [8]:
def avg_turn_length(text):
    if not isinstance(text, str): return 0
    turns = re.split(r"(?:^|\s)(?:[A-ZĐƠÔƯỮẠ-ỹa-z]+:)", text)
    turns = [t.strip() for t in turns if t.strip()]
    word_counts = [len(word_tokenize(t)) for t in turns]
    return round(sum(word_counts) / len(word_counts), 1) if word_counts else 0

df["avg_turn_length"] = df["Original Text"].apply(avg_turn_length)

In [9]:
df.head(5)

,Original Text,Label,Emotional Reasoning,Overgeneralization,Should Statements,Personalization,Mental Filter,Disqualifying the Positive,Jumping to Conclusions,Labeling and Mislabeling,Magnification and Minimization,All-or-Nothing Thinking,turn_count,question_ratio,discourse_marker_count,self_reference_count,avg_turn_length
0,"Cô ấy luôn có những hành động kỳ lạ, ví dụ như...",0,0,0,0,0,0,0,0,0,0,0,1,1.000,0,4,79.0
1,"Nếu bố mẹ tôi biết tôi đang vật lộn thế nào, h...",1,0,0,0,0,0,0,1,0,0,0,1,1.000,0,4,30.0
2,"Trong năm thứ 4 và thứ 5 của sự nghiệp, tôi kh...",1,0,0,0,0,0,0,0,0,0,1,1,1.000,0,6,78.0
3,"Tôi muốn bị ốm, và tôi biết điều đó thật khủng...",0,0,0,0,0,0,0,0,0,0,0,1,1.000,1,4,19.0
4,Trước khi chúng tôi bắt đầu hẹn hò và trong nă...,0,0,0,0,0,0,0,0,0,0,0,1,0.333,0,2,39.0


In [10]:
df.to_csv("Interaction_features.csv", index=False)